In [1]:
import pandas as pd
import geopandas as gpd
from h3 import h3
import keplergl
from shapely.geometry import Polygon
import os
from glob import glob

In [6]:
class HexBins():

    def coordsToH3HexBinID(self, row):
        return h3.geo_to_h3(row['LAT'], row['LON'], self.h3Level)
    
    def addGeometry(self, row):
        points = h3.h3_to_geo_boundary(row['HexID'], True)
        return Polygon(points)
    
    def __init__(self,directory):
        currentDirectory = os.chdir(directory)
        dataFiles = glob('*.csv')
        self.h3Level = 10
        dfList = []
        for file in dataFiles:
            df = pd.read_csv(file)
            columns = ['CITY_TOWN_NAME','CRASH_DATE_TEXT','CRASH_TIME','LAT','LON']
            df.drop(df.columns.difference(columns), 1, inplace=True)
            df.dropna(inplace=True)
            dfList.append(df)
        data = pd.concat(dfList)
        data['H3Index'] = data.apply(self.coordsToH3HexBinID, axis=1)
        groupedDF = data.groupby(by='H3Index').size().reset_index()
        groupedDF.columns = ['HexID', 'Count']
        groupedDF['geometry'] = groupedDF.apply(self.addGeometry, axis=1)
        self.geoDF = gpd.GeoDataFrame(groupedDF)
        return
    
    def mapData(self):
        keplerMap = keplergl.KeplerGl(height = 800)
        keplerMap.add_data(data = self.geoDF, name = 'AccidentClusters')
        keplerMap
        

In [7]:
test = HexBins(r"C:\Users\Kevin\Documents\AccidentData")

C:\Users\Kevin\anaconda3\envs\gdal\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (25,29,30,32,33,34,67,109) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\Kevin\anaconda3\envs\gdal\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (25,29,30,32,33,67) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [18]:
keplerMap = keplergl.KeplerGl(height = 400)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [22]:
test.geoDF.isnull()

,HexID,Count,geometry
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
78956,False,False,False
78957,False,False,False
78958,False,False,False
78959,False,False,False


In [19]:
keplerMap.add_data(test.geoDF)

AttributeError: 'str' object has no attribute '_geom'